In [7]:
!pip install pytest-mock

In [1]:
!pip install pytest pytest-asyncio


In [11]:
%%writefile cashbook.py
import asyncio
import threading

# Mock classes

class Cash:
    def __init__(self, symbol, amount, conversion_rate):
        self.symbol = symbol
        self.amount = amount
        self.conversion_rate = conversion_rate
        self.updated_callbacks = []

    async def set_amount(self, amount):
        self.amount = amount
        await self._trigger_update()

    async def _trigger_update(self):
        for callback in self.updated_callbacks:
            await callback(self)

    def on_update(self, callback):
        self.updated_callbacks.append(callback)


class CashBookUpdatedEventArgs:
    def __init__(self, update_type, cash):
        self.update_type = update_type
        self.cash = cash


# Main CashBook Class
class CashBook:
    def __init__(self):
        self._currencies = {}
        self._account_currency = "USD"

    @property
    def account_currency(self):
        return self._account_currency

    @account_currency.setter
    def account_currency(self, value):
        if self._account_currency in self._currencies:
            amount = self._currencies[self._account_currency].amount
            del self._currencies[self._account_currency]
        else:
            amount = 0

        self._account_currency = value.upper()
        self._currencies[self._account_currency] = Cash(self._account_currency, amount, 1.0)

    @property
    def total_value_in_account_currency(self):
        total_value = 0
        for symbol, cash in self._currencies.items():
            total_value += cash.amount * cash.conversion_rate
        return total_value

    async def add(self, symbol, quantity, conversion_rate):
        cash = Cash(symbol, quantity, conversion_rate)
        await self._add_internal(symbol, cash)

    async def _add_internal(self, symbol, cash):
        if symbol in self._currencies:
            existing_cash = self._currencies[symbol]
            existing_cash.conversion_rate = cash.conversion_rate
            await existing_cash.set_amount(cash.amount)
            action = "updated"
        else:
            cash.on_update(self._on_cash_update)
            self._currencies[symbol] = cash
            action = "added"

        await self._notify_update(action, cash)

    async def remove(self, symbol):
        if symbol in self._currencies:
            cash = self._currencies.pop(symbol)
            cash.on_update(lambda _: None)
            await self._notify_update("removed", cash)

    async def _notify_update(self, action, cash):
        event_args = CashBookUpdatedEventArgs(action, cash)
        print(f"Cash {event_args.update_type}: {event_args.cash.symbol} - Amount: {event_args.cash.amount}, Conversion Rate: {event_args.cash.conversion_rate}")

    async def _on_cash_update(self, cash):
        await self._notify_update("updated", cash)


    def convert(self, source_quantity, source_currency, destination_currency):
        if source_quantity == 0:
            return 0

        if source_currency not in self._currencies or destination_currency not in self._currencies:
            raise ValueError("Conversion rate not found for one of the currencies.")

        source = self._currencies[source_currency]
        destination = self._currencies[destination_currency]

        if source.conversion_rate == 0 or destination.conversion_rate == 0:
            raise ValueError("Conversion rate is zero for one of the currencies.")

        conversion_rate = source.conversion_rate / destination.conversion_rate
        return source_quantity * conversion_rate


    def convert_to_account_currency(self, source_quantity, source_currency):
        if source_currency == self.account_currency:
            return source_quantity
        return self.convert(source_quantity, source_currency, self.account_currency)


    async def ensure_currency_data_feeds(self, required_currencies, subscriptions):
          added_feeds = []
          for currency in required_currencies:
              if currency not in self._currencies:
                  await self.add(currency, 0, 1.0)
                  added_feeds.append(currency)


              if currency not in subscriptions:
                  subscriptions.add(currency)
                  if currency not in added_feeds:
                      added_feeds.append(currency)

          return added_feeds






Overwriting cashbook.py


In [12]:
%%writefile test_cashbook.py
import pytest
from unittest.mock import AsyncMock
from cashbook import Cash, CashBook

@pytest.fixture
def cash_book():
    return CashBook()

@pytest.mark.asyncio
async def test_add_currency(cash_book, mocker):
    mock_notify_update = AsyncMock()
    mocker.patch.object(cash_book, "_notify_update", mock_notify_update)

    await cash_book.add("EUR", 1000, 0.85)
    assert "EUR" in cash_book._currencies
    assert cash_book._currencies["EUR"].amount == 1000
    assert cash_book._currencies["EUR"].conversion_rate == 0.85
    mock_notify_update.assert_called_once_with("added", cash_book._currencies["EUR"])

@pytest.mark.asyncio
async def test_remove_currency(cash_book, mocker):
    mock_notify_update = AsyncMock()
    mocker.patch.object(cash_book, "_notify_update", mock_notify_update)

    await cash_book.add("GBP", 500, 0.75)
    cash_to_remove = cash_book._currencies["GBP"]
    await cash_book.remove("GBP")
    assert "GBP" not in cash_book._currencies
    mock_notify_update.assert_called_with("removed", cash_to_remove)

@pytest.mark.asyncio
async def test_update_currency(cash_book, mocker):
    mock_notify_update = AsyncMock()
    mocker.patch.object(cash_book, "_notify_update", mock_notify_update)

    await cash_book.add("USD", 2000, 1.0)
    assert mock_notify_update.call_count == 1

    await cash_book.add("USD", 1500, 1.1)
    assert cash_book._currencies["USD"].amount == 1500
    assert cash_book._currencies["USD"].conversion_rate == 1.1
    assert mock_notify_update.call_count == 3

@pytest.mark.asyncio
async def test_total_value_in_account_currency(cash_book, mocker):
    mock_notify_update = AsyncMock()
    mocker.patch.object(cash_book, "_notify_update", mock_notify_update)

    await cash_book.add("EUR", 1000, 0.85)
    await cash_book.add("USD", 2000, 1.0)
    expected_total = 1000 * 0.85 + 2000 * 1.0
    assert cash_book.total_value_in_account_currency == expected_total

@pytest.mark.asyncio
async def test_on_update_called(cash_book, mocker):
    cash = Cash("JPY", 10000, 0.009)
    mock_callback = AsyncMock()
    cash.on_update(mock_callback)

    mocker.patch.object(cash_book, "_notify_update", AsyncMock())
    await cash_book.add("JPY", 10000, 0.009)

    await cash.set_amount(20000)

    mock_callback.assert_called_once_with(cash)

@pytest.mark.asyncio
async def test_cashbook_notify_update_called(cash_book, mocker):
    mock_notify_update = AsyncMock()
    mocker.patch.object(cash_book, "_notify_update", mock_notify_update)

    await cash_book.add("USD", 1000, 1.0)
    mock_notify_update.assert_called_once_with("added", cash_book._currencies["USD"])

@pytest.mark.asyncio
async def test_remove_currency_calls_notify_update(cash_book, mocker):
    await cash_book.add("EUR", 1000, 0.85)
    mock_notify_update = AsyncMock()
    mocker.patch.object(cash_book, "_notify_update", mock_notify_update)

    cash_to_remove = cash_book._currencies["EUR"]

    await cash_book.remove("EUR")

    mock_notify_update.assert_called_once_with("removed", cash_to_remove)

@pytest.mark.asyncio
async def test_account_currency(cash_book, mocker):
    mocker.patch.object(cash_book, "_notify_update", AsyncMock())

    cash_book.account_currency = "EUR"
    assert cash_book.account_currency == "EUR"
    assert cash_book._currencies["EUR"].amount == 0
    await cash_book.add("EUR", 1000, 0.85)
    assert cash_book.total_value_in_account_currency == 1000 * 0.85



def test_convert(cash_book):
    cash_book._currencies["EUR"] = Cash("EUR", 1000, 0.85)
    cash_book._currencies["USD"] = Cash("USD", 2000, 1.0)

    result = cash_book.convert(1000, "EUR", "USD")
    assert result == 1000 * 0.85 / 1.0

    result_reverse = cash_book.convert(1000, "USD", "EUR")
    assert result_reverse == 1000 * 1.0 / 0.85

def test_convert_to_account_currency(cash_book):
    cash_book._currencies["EUR"] = Cash("EUR", 1000, 0.85)
    cash_book._currencies["USD"] = Cash("USD", 2000, 1.0)


    cash_book.account_currency = "USD"

    result = cash_book.convert_to_account_currency(1000, "EUR")
    assert result == 1000 * 0.85

    result_same_currency = cash_book.convert_to_account_currency(1000, "USD")
    assert result_same_currency == 1000

@pytest.mark.asyncio
async def test_ensure_currency_data_feeds(cash_book, mocker):
    mock_notify_update = AsyncMock()
    mocker.patch.object(cash_book, "_notify_update", mock_notify_update)

    subscriptions = set()
    required_currencies = ["EUR", "JPY"]


    added_feeds = await cash_book.ensure_currency_data_feeds(required_currencies, subscriptions)

    assert "EUR" in cash_book._currencies
    assert "JPY" in cash_book._currencies
    assert "EUR" in subscriptions
    assert "JPY" in subscriptions
    assert added_feeds == ["EUR", "JPY"]


    await cash_book.add("USD", 1000, 1.0)
    required_currencies.append("USD")
    added_feeds = await cash_book.ensure_currency_data_feeds(required_currencies, subscriptions)

    assert "USD" in subscriptions
    assert added_feeds == ["USD"]




Overwriting test_cashbook.py


In [13]:
!pytest test_cashbook.py --asyncio-mode=auto


/usr/local/lib/python3.10/dist-packages/pytest_asyncio/plugin.py:208: PytestDeprecationWarning: The configuration option "asyncio_default_fixture_loop_scope" is unset.
The event loop scope for asynchronous fixtures will default to the fixture caching scope. Future versions of pytest-asyncio will default the loop scope for asynchronous fixtures to function scope. Set the default fixture loop scope explicitly in order to avoid unexpected behavior in the future. Valid fixture loop scopes are: "function", "class", "module", "package", "session"

  warnings.warn(PytestDeprecationWarning(_DEFAULT_FIXTURE_LOOP_SCOPE_UNSET))
======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-8.3.3, pluggy-1.5.0
rootdir: /content
plugins: asyncio-0.24.0, mock-3.14.0, anyio-3.7.1, typeguard-4.3.0
asyncio: mode=auto, default_loop_scope=None
collected 11 items                                                                  